# W266 Final Project Baseline

In [11]:
# Import libraries.
import re
import numpy as np
import json
import pandas as pd
import sys
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

In [2]:
# Load the data
csv_file = 'data/yelp_academic_dataset_review.csv'
review_data = pd.read_csv(csv_file)

In [12]:
review_count = len(review_data.index)
print 'The dataset contains %d reviews.' % review_count
print 'The dataset columns:'
print review_data.columns

The dataset contains 2685066 reviews.
The dataset columns:
Index([u'user_id', u'review_id', u'text', u'votes.cool', u'business_id',
       u'votes.funny', u'stars', u'date', u'type', u'votes.useful'],
      dtype='object')


In [4]:
# Create training dataset and test dataset

reviews = review_data['text'].tolist()
stars = review_data['stars'].tolist()
test_data, test_labels = reviews[:review_count/3], stars[:review_count/3]
train_data, train_labels = reviews[review_count/3:], stars[review_count/3:]
print 'The training dataset contains %d review.' %(len(train_data))
print 'The test dataset contains %d review.' %(len(test_data))
#print train_data[0], train_labels[0]
print test_data[0], test_labels[0]

The training dataset contains 1790044 review.
The test dataset contains 895022 review.
Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road. 4


In [13]:
# Split training dataset to dev dataset and mini-train dataset

train_count = len(train_data)
dev_data, dev_labels = train_data[train_count/2:], train_labels[train_count/2:]
mini_train_data, mini_train_labels = train_data[:train_count/2], train_labels[:train_count/2]

print 'The mini training dataset contains %d samples.' %(len(mini_train_labels))
print 'The dev dataset contains %d samples.' %(len(dev_labels)) 

print mini_train_data[0], mini_train_labels[0]

The mini training dataset contains 895022 samples.
The dev dataset contains 895022 samples.
A requirement of my job as a retail manager is to renew my State of Nevada Health Card and Alcohol Awareness handling card every four years. The first thing I want to bring up is that there is a MAJOR difference between the Southern Nevada Health District office (SNHD) in Henderson and the Alcohol Awareness Training office off Sunset Road in Henderson. Where the SNHD gave me the IMPRESSION of being both an inconvenient tax and  socialism in the shadow of the Las Vegas Strip, spending time in the Alcohol Awareness Training class was enjoyable. Thanks to our trainer Steve which I will yelp about soon. 

 A major advantage of this Alcohol Awareness Training office is that it is by Sunrise Coffee. I am the type of person that strives to get to an appointment or Yelp event early. So, having Sunrise Coffee a few steps away was nice. I parked my car, relaxed with a cup of coffee and when it was near th

In [14]:
# Train the MultinomialNB model and get the best alpha value.

print 'Word Vectorizer - MultinomialNB'
word_vectorizer = CountVectorizer()  #define and fit the model
vect_mini_train = word_vectorizer.fit_transform(mini_train_data)
vect_dev = word_vectorizer.transform(dev_data)
print 'Word vocabulary size:',vect_mini_train.shape[1]  #display word vocabulary size

a = 0.0
best_f1 = 0
best_alpha = 0
while a <= 2.0:
    mnb = MultinomialNB(alpha = a)
    mnb.fit(vect_mini_train, mini_train_labels)
    preds = mnb.predict(vect_dev)
    f1_score = metrics.f1_score(dev_labels, preds, average='weighted')
    if f1_score > best_f1:
        best_f1 = f1_score
        best_alpha = a
    a += 0.1

print 'Best alpha: ', best_alpha


Word Vectorizer - MultinomialNB
Word vocabulary size: 263843
Best alpha:  1.6


In [15]:
# Train with all training dataset.

word_vectorizer = CountVectorizer()
vect_train = word_vectorizer.fit_transform(train_data)
vect_test = word_vectorizer.transform(test_data)

mnb = MultinomialNB(alpha = 1.6)
mnb.fit(vect_train, train_labels)

# Predict stars of test reveiws
preds = mnb.predict(vect_test)
pred_prob = mnb.predict_proba(vect_test)

In [18]:
f1 = metrics.f1_score(test_labels, preds, average='weighted')
accuracy = metrics.accuracy_score(test_labels, preds)
print "Accuracy: %.03f" % accuracy
print "F1 score: %.03f" % f1

Accuracy: 0.568
F1 score: 0.564
